In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Example of comparative analysis for uCT and CBCT images

This file perform the complete analysis for a uCT and a CBCT images. A selected number of random RoIs will be extracted from both images. Trabeculae will be segmented using the BoneJ segmentation algorithm that search for a global threshold over the RoI and maximizes the connectivity. Once RoIs are extracted, metrics using BoneJ will be extracted and saved into a CSV file for further analysis and comparison.

### Preregistration

If the images are not registered before analysis, they will be registered using elastix registration package. Due to the big size of the uCT image, a subsampled version of the uCT will be registered instead the original one because memory limitations in the computer.

#### Initial Registration

Currently, in order to speed up the registration performance and ensure a correct solution, a preregistration is performed using 3DSlicer following the next steps: 

1. Load uCT subsampled image 
2. Load CBCT image
3. Create a transformation and apply to uCT (registration transforms uCT to the CBCT geometrical space)
4. Play to align both images as much as you can
5. Save the registration parameters as a *.tfm file into the corresponding folder

### Image format

All images will be saved and read as nifti format. For transforming your images you will find functions for MATLAB in the corresponding folder. Also, if you are using ImageJ, a plugin that deal with nifti is also available in the corresponding folder

### 


### <font color='red'>Variables that need to be changed </font>

- <font color='red'>Make sure that the BoneJ and NiftiIO plugins has been installed into imagej used version

In [2]:
import os
# Python packages
PythonTools_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\PythonTools'
pyimagej_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\pyimagej'
TrabeculaeTools_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\TrabeculaeTools'
elastix_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Programs\\elastix'
ImageJ_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Programs\\Fiji'
Macros_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\TrabeculaeTools\\BoneJMacros'

# Elastix registration
elastix_EXE = os.path.join(elastix_DIR,'elastix.exe')

# Path to imageJ (Need BoneJ and NII reader plugins)
ImageJ_EXE = os.path.join(ImageJ_DIR,'ImageJ-win64.exe')

# Macro definition for segmentation Trabeculae
SegmentfileDescription_XML = os.path.join(Macros_DIR,'SegmentTrabeculaImageJMacro.xml')
Segmentmacro_IJM = os.path.join(Macros_DIR,'SegmentTrabeculaImageJMacro.ijm')

# Macro definition for Metric Trabeculae
MetricfileDescription_XML= os.path.join(Macros_DIR,'BoneJMetricsImageJMacro.xml')
Metricmacro_IJM = os.path.join(Macros_DIR,'BoneJMetricsImageJMacro.ijm')

## Import need packages

In [3]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

# Add Python Tools to path
sys.path.append(PythonTools_DIR)

# Add pyimagej
sys.path.append(pyimagej_DIR)

# Add TrabeculaeTools
sys.path.append(TrabeculaeTools_DIR)

In [15]:
# PythonTools imports
from PythonTools.helpers.elastix import elastix as elastixhelper
from PythonTools import io, transformations

# pyimagej imports
from pyimagej.pyimagej import ImageJ, MacroImageJ

# TrabeculaeTools imports
from TrabeculaeTools.analysis import DownsampleImage, CreateRoIfileStudy, CreateRoI, CreateRoITransformed, PrintPercentage
from TrabeculaeTools.analysis import SegmentTrabeculaeBoneJ, GetResultsFromSegmentation, GetResultsForComparison

## Parameters of study

This will determine the complete analysis to be performed

- RandomSeed select the random seed for the creation of RoIs and other random variables. If you leave does not change it, your results will be allways the same.

- RoISizeVector is the vector of different sizes of RoI to be used

- NRandomRoIs is the number of RoIs that will be extracted for each RoI Size

In [5]:
# To do it repetible
RandomSeed = 2

# Size of RoIs that will be studied in mm
RoISizeVector = [2.5]# np.arange(2.2,5,0.2) # If you want to use only one size use this -> RoISizeVector = [4.4] for a RoI size of 4.4mm

# Number of RoIs that will be generated for each RoI Size
NRandomRoIs = 1

# Path for analysis results, input images and outputs
studyFolder ='J:\\Projects\\JHUTrabeculae\\Results\\uCTvsCBCT_example'

## Define the images for analysis and parameters for the study

The following folder structure <font color='red'>must be stablished</font> before analysis. 


### Data Folder Structure
- :open_file_folder: studyPath
  - :file_folder: Originals
    - :page_with_curl: CBCT.nii (Target image of CBCT)
    - :page_with_curl: uCT.nii (Source image of uCT)
    - :page_with_curl: CBCT_BoneMask.nii (Total Bone Segmentation of CBCT image)
  - :file_folder: Registration
    - :page_with_curl: RegistrationParams.txt (elastix parameters for registration)



In [7]:
# Starting Folders
originalsFolder = os.path.join(studyFolder,'Originals') 
registrationFolder = os.path.join(studyFolder,'Registration')

# Original Images for analysis
CBCTimagePath = os.path.join(originalsFolder,'CBCT.nii')
uCTimagePath  = os.path.join(originalsFolder,'uCT.nii')

# Total Bone of CBCT image
BoneCBCTimagePath = os.path.join(originalsFolder,'CBCT_BoneMask.nii')

# Folder For RoI results
RoiFolder = os.path.join(studyFolder,'RoIFiles')

# 1. Preregistration

In order to help registration performance, we first semi align the images using Slicer. The transformation will be saved as a *.tmf file into Registration folder using the name "InitialTransform.tfm"

In [7]:
# Perform the initial registration for image uCT
InitialTransform = os.path.join(registrationFolder,'InitialTransform.tfm')

# 2. Perform Registration

Usually, uCT image is quite big and memory cannot handle it. To facilitate the registration process, we create a downsampled version of uCT. This image will be registered to the actual CBCT image and registration parameters will be saved for further use.

In [8]:
registrationParametersPath = os.path.join(registrationFolder,'Registrationparams.txt')

   ## 2.1 Downsampling

In [9]:
DownsamplingFactor = 3
DownsampleduCTimagePath = os.path.join(registrationFolder,'uCTDownsampled.nii')

# This uses Linear Interpolation. BsplineInterpolator cannot handle the resampling.
DownsampleImage(uCTimagePath,DownsampleduCTimagePath,DownsamplingFactor)

[36.07 s]    - Downsampling...
[71.36 s]    - Finished!


 ## 2.2 Registration

In [10]:
# Transform Initial Transform to elastix format
InitialTransformElastix = os.path.join(registrationFolder,'InitialTransform.txt')
elastixhelper(elastix_EXE).itk2elastix(InitialTransform,InitialTransformElastix)

# Create Command line Registration
cmdString = elastix_EXE + \
            ' -f ' + CBCTimagePath + \
            ' -m ' + DownsampleduCTimagePath + \
            ' -p ' + registrationParametersPath + \
            ' -t0 ' + InitialTransformElastix + \
            ' -out '+ registrationFolder + \
            ' -threads 4'
            
# Launch Registration
os.system("start /wait cmd /c " + cmdString)

0

In [9]:
# Join Transformation and create final transformation
RegistrationTransformElastix = os.path.join(registrationFolder,'TransformParameters.0.txt')

RegistrationTransform = os.path.join(registrationFolder,'TransformParameters.tfm')
elastixhelper(elastix_EXE).elastix2itk(RegistrationTransformElastix,RegistrationTransform)

# Compose Initial and Registration transformation
RegistrationTransformFinal = os.path.join(registrationFolder,'RegistrationTransformFinal.tfm')

Tinitial = io.load_tfm(InitialTransform)
TReg = io.load_tfm(RegistrationTransform)
TRegistrationComplete = transformations.concatenate_matrices(TReg, Tinitial)

# Save the resulting transformation into file
io.save_tfm(RegistrationTransformFinal,TRegistrationComplete)

NameError: name 'InitialTransform' is not defined

# 3. Generate RoI file for analysis

This file contains the RoI definition that will be analyzed for both images.

In [12]:
# Folder for RoI file save
CreateRoIfileStudy(BoneCBCTimagePath, RoiFolder, RoISizeVector, NRandomRoIs, RandomSeed, PrintDebug = False)

Creating RoI File: [16:39:35]--[||||||||||||||||||||] Finished!

# 4. Segmentation, Metric Analysis and Save Results

In [16]:
#########################
# Path To RoI
RegistrationTransformFinal = os.path.join(registrationFolder,'RegistrationTransformFinal.tfm')
PathToRoIfile = os.path.join(RoiFolder,'RoIFileComplete.csv')

# Read RoI definitions
RoIStructure = pd.read_csv(PathToRoIfile,index_col=0)
NumberOfRoIs = len(RoIStructure.index)

# Folder for results
ResultsDir = os.path.join(studyFolder,'Results')
if not os.path.exists(ResultsDir):
    os.makedirs(ResultsDir)

# Temporal folder for results
MetricsOutputDir = os.path.join(studyFolder,'MetricOutputTemp')
if not os.path.exists(MetricsOutputDir):
    os.makedirs(MetricsOutputDir)

percentage = 0
for i in range(NumberOfRoIs):
    RoIDefinition = RoIStructure.iloc[i]
    
    #####
    ##### ROI Extraction
    #####
    
    # Extract RoI from CBCT
    PrintPercentage(percentage, preMessage = 'Extracting RoI from CBCT                             ')

    RoIFilePathCBCT = os.path.join(RoiFolder,CBCTimagePath.split('\\')[-1][:-4] + '_{1:.2f}mm_RoI{0:d}.nii'.format(RoIDefinition['RoI Number'],RoIDefinition['RoI Size mm']))
    
    CreateRoI(  ImageFilePath = CBCTimagePath,\
                RoIDefinition = RoIDefinition,\
                RoIFilePath = RoIFilePathCBCT, \
                PrintDebug = False)

    # Extract RoI from uCT
    PrintPercentage(percentage, preMessage = 'Extracting RoI from uCT                              ')
    
    RoIFilePathuCT = os.path.join(RoiFolder,uCTimagePath.split('\\')[-1][:-4] + '_{1:.2f}mm_RoI{0:d}.nii'.format(RoIDefinition['RoI Number'],RoIDefinition['RoI Size mm']))

    CreateRoITransformed(ImageFilePath = uCTimagePath,\
                         RoIDefinition = RoIDefinition,\
                         TransformationFile = RegistrationTransformFinal,\
                         ReferenceRoIImageFilePath = RoIFilePathCBCT,\
                         RoIFilePath = RoIFilePathuCT,\
                         PrintDebug = False)
    
    #####
    ##### ROI Segmentation
    #####
    
    
    # Segmentation for CBCT
    PrintPercentage(percentage, preMessage = 'Segmenting CBCT Using BoneJ                          ')
    
    RoIFilePathCBCTSegmented = os.path.join(RoiFolder,CBCTimagePath.split('\\')[-1][:-4] + '_{1:.2f}mm_RoI{0:d}_SegmentedBoneJ.nii'.format(RoIDefinition['RoI Number'],RoIDefinition['RoI Size mm']))

    CBCTdataSegmentation = SegmentTrabeculaeBoneJ(  ImageJ_EXE, Segmentmacro_IJM, SegmentfileDescription_XML, 
                                                    defaultTimeout = 700,\
                                                    PathToRoIfile = RoIFilePathCBCT,\
                                                    PathToSegmentedRoIfile = RoIFilePathCBCTSegmented,\
                                                    SMOOTH_Sigma = 0.03,\
                                                    TH_Erosion = 0,\
                                                    TH_Dilation = 0)
    
    # Segmentation for CBCT
    PrintPercentage(percentage, preMessage = 'Segmenting uCT Using BoneJ                           ')
    
    RoIFilePathuCTSegmented = os.path.join(RoiFolder,uCTimagePath.split('\\')[-1][:-4] + '_{1:.2f}mm_RoI{0:d}_SegmentedBoneJ.nii'.format(RoIDefinition['RoI Number'],RoIDefinition['RoI Size mm']))

    uCTdataSegmentation = SegmentTrabeculaeBoneJ(   ImageJ_EXE, Segmentmacro_IJM, SegmentfileDescription_XML, 
                                                    defaultTimeout = 700,\
                                                    PathToRoIfile = RoIFilePathuCT,\
                                                    PathToSegmentedRoIfile = RoIFilePathuCTSegmented,\
                                                    SMOOTH_Sigma = 0.03,\
                                                    TH_Erosion = 0,\
                                                    TH_Dilation = 0)
    
    #####
    ##### ROI ANALYSIS
    #####
    
    
    
    BoneJMetrics = MacroImageJ(imagejPath = ImageJ_EXE,\
                               macroPath = Metricmacro_IJM,\
                               xmlDefinition = MetricfileDescription_XML,\
                               defaultTimeout = 700)
    
    ANISOTROPY_Radius = 0.9 * RoIDefinition['RoI Size mm']/2.0
    #try:
    # Analysis for CBCT
    PrintPercentage(percentage, preMessage = 'Calculating Metrics for CBCT Segmented with BoneJ    ')

    params = BoneJMetrics.runMacro( inputImage = RoIFilePathCBCTSegmented,\
                                    outputDir = MetricsOutputDir,\
                                    ANISOTROPY_Radius = ANISOTROPY_Radius)
    CBCTdataMetricsParameters = pd.DataFrame([params], index = [0])
    CBCTDataResults = GetResultsFromSegmentation(CBCTdataSegmentation, CBCTdataMetricsParameters)



    # Analysis for uCT
    PrintPercentage(percentage, preMessage = 'Calculating Metrics for uCT Segmented with BoneJ     ')

    params = BoneJMetrics.runMacro( inputImage = RoIFilePathuCTSegmented,\
                                    outputDir = MetricsOutputDir,\
                                    ANISOTROPY_Radius = ANISOTROPY_Radius)
    uCTdataMetricsParameters = pd.DataFrame([params], index = [0])
    uCTDataResults = GetResultsFromSegmentation(uCTdataSegmentation, uCTdataMetricsParameters)
    
    #####
    ##### ROI Comparison
    #####
    
    PrintPercentage(percentage, preMessage = 'Generating Results for CBCT-uCT Segmented with BoneJ ')

    FinalResults = GetResultsForComparison(uCTDataResults, CBCTDataResults,\
                                           RoISize = RoIDefinition['RoI Size mm'],\
                                           RoINumber = RoIDefinition['RoI Number'],\
                                           RoIX = RoIDefinition['Center x mm'],\
                                           RoIY = RoIDefinition['Center y mm'],\
                                           RoIZ = RoIDefinition['Center z mm'])


    # Save results

    FileResults = os.path.join(ResultsDir, CBCTimagePath.split('\\')[-1][:-4] + '_{1:.2f}mm_RoI{0:d}_Results.csv'.format(RoIDefinition['RoI Number'],RoIDefinition['RoI Size mm']))
    FinalResults.to_csv(FileResults)
    
    #except:
    #    pass
    
    PrintPercentage(percentage, preMessage = 'Finished one RoI from file                           ')
    percentage = 100.0 * float(i) / (NumberOfRoIs-1)
    PrintPercentage(percentage, preMessage = 'Complete Analysis ')




Calculating Metrics for CBCT Segmented with BoneJ    [17:27:54]--[                    ]Finish!, total time:  0.04
Calculating Metrics for uCT Segmented with BoneJ     [17:28:13]--[                    ]Finish!, total time:  0.04
Calculating Metrics for CBCT Segmented with BoneJ    [17:32:03]--[                    ]Finish!, total time:  0.04
Calculating Metrics for uCT Segmented with BoneJ     [17:33:13]--[                    ]Finish!, total time:  0.05
Complete Analysis [17:35:12]--[||||||||||||||||||||] Finished!

# 5. Read Results and Join under one matrix data

Results generate a csv file for each RoI analysis. This will read all of them and join together under one matrix

In [22]:
# Read all files and join
datalist = list()
for f in os.listdir(ResultsDir):
    newData = pd.read_csv(os.path.join(ResultsDir,f),index_col = 0)
    datalist.append(newData)

datamatrix = datalist[0]
for i in range(1,len(datalist)):
    datamatrix = pd.concat([datamatrix, datalist[i]])
    
# Create folder for saving File and Graph
graphResultFolder = os.path.join(studyFolder,'FinalResult_and_Graphs')
if not os.path.exists(graphResultFolder):
        os.makedirs(graphResultFolder)
        
# Save file
datamatrix.to_csv(graphResultFolder + r'\Data.csv')

# 6. Plot Results

## 6.1 Print Results Comparison uCT Vs CBCT Relative Error

## 6.2 Print Results uCT Vs distance metrics

## 6.3 Print Results uCT Vs CBCT for Jaccard and Dice

## 6.4 Print Results uCT Vs RoI localization

## 6.5 Print Results uCT vs CBCT